In [59]:
import numpy as np
import pandas as pd
import graphlab
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold

BASE  = './bytecup2016data'
IINFO = BASE + '/invited_info_train.txt'
VAL   = BASE + '/validate_nolabel.txt'
TEST   = BASE + '/test_nolabel.txt'

invdata = pd.read_csv(IINFO, delim_whitespace=True, header=None, names=["qid", "uid", "label"])
valdata = pd.read_csv(VAL)

In [62]:
def algorithm_predict(new_invdata, new_valdata):
    sf = graphlab.SFrame(new_invdata, format='dataframe')
    #m = graphlab.recommender.create(sf, target='rating')
    m = graphlab.item_similarity_recommender.create(sf, user_id='qid', item_id='uid', 
                                                                      target='label')
    #m = graphlab.factorization_recommender.create(sf,target='rating', num_factors=32, regularization=1e-9, 
                                                  #linear_regularization=1e-09, max_iterations=500)
    #m = graphlab.recommender.ranking_factorization_recommender.create(sf, user_id='qid', item_id='uid', 
                                                                      #target='label')
    #m = graphlab.popularity_recommender.create(sf, target='rating')  
    
    sf = graphlab.SFrame(new_valdata, format='dataframe')
    return m.predict(sf)

In [63]:
skf = StratifiedKFold(n_splits = 5)
invdata_label = invdata['label']
count = 1
for train_invdata, test_valdata in skf.split(invdata,invdata_label):
    new_invdata = invdata.loc[train_invdata]
    new_valdata = invdata.loc[test_valdata]
    new_invdata.reset_index(drop = True, inplace = True)
    new_valdata.reset_index(drop = True, inplace = True)
    
    new_valdata['label'] = algorithm_predict(new_invdata, new_valdata)
    
    new_valdata.to_csv('cross-validation-attempt' + str(count) + '.csv', separator=",")
    count += 1

Recsys training: model = item_similarity

Preparing data set.

Data has 196601 observations with 7572 users and 25692 items.

Data prepared in: 0.593923s

Training model from provided data.

...

| 7.74s                               | 100              | 25623           |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 7.78068s